## Interval Analysis

In [1]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=-1) # added softmax for probabilities
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()


Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [2]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [3]:
train_model(model, 15)
test_model(model)

Epoch 1/15, Loss: 2.012
Epoch 2/15, Loss: 1.729
Epoch 3/15, Loss: 1.627
Epoch 4/15, Loss: 1.596
Epoch 5/15, Loss: 1.582
Epoch 6/15, Loss: 1.572
Epoch 7/15, Loss: 1.566
Epoch 8/15, Loss: 1.561
Epoch 9/15, Loss: 1.557
Epoch 10/15, Loss: 1.553
Epoch 11/15, Loss: 1.550
Epoch 12/15, Loss: 1.547
Epoch 13/15, Loss: 1.545
Epoch 14/15, Loss: 1.542
Epoch 15/15, Loss: 1.540
Accuracy on images: 93.43


### Write the interval analysis for the simple model

In [4]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation

In [5]:
!pip install bound_propagation

In [19]:
import torch
from torch import nn
from bound_propagation import BoundModule, IntervalBounds, BoundModelFactory

class BoundNormalize(BoundModule):
    def __init__(self, model, factory, **kwargs):
        super().__init__(model, factory, **kwargs)
        self.mean = torch.as_tensor(getattr(model, "mean", 0.1307))
        self.std  = torch.as_tensor(getattr(model, "std", 0.3081))

    def propagate_size(self, in_size):
        self._in_size = in_size
        return in_size

    @property
    def need_relaxation(self):
        return False

    def clear_relaxation(self):
        pass

    def _reshape_params(self, x):
        mean = self.mean.to(x.device, x.dtype)
        std  = self.std.to(x.device, x.dtype)
        if mean.ndim == 1 and x.ndim >= 2 and mean.numel() == x.size(1):
            view = [1, x.size(1)] + [1] * (x.ndim - 2)
            mean = mean.view(*view)
            std  = std.view(*view)
        return mean, std

    def ibp_forward(self, bounds, save_relaxation=False, save_input_bounds=False):
        lo, hi = bounds.lower, bounds.upper
        mean, std = self._reshape_params(lo)
        scale = 1.0 / std
        bias  = -mean / std
        lo_ = torch.minimum(scale * lo, scale * hi) + bias
        hi_ = torch.maximum(scale * lo, scale * hi) + bias
        return IntervalBounds(bounds.region, lo_, hi_)

    def crown_backward(self, *args, **kwargs):
        raise NotImplementedError("not impl")

class BoundNet(BoundModule):
    def __init__(self, model, factory, **kwargs):
        super().__init__(model, factory, **kwargs)
        self.fc  = model.fc
        self.fc2 = model.fc2

    def propagate_size(self, in_size):
        return (self.fc2.out_features,)

    @property
    def need_relaxation(self):
        return False

    def clear_relaxation(self):
        pass

    @staticmethod
    def _linear_ibp(l, u, weight, bias):
        W = weight
        W_pos = torch.clamp(W, min=0)
        W_neg = torch.clamp(W, max=0)
        lower = l.matmul(W_pos.t()) + u.matmul(W_neg.t())
        upper = u.matmul(W_pos.t()) + l.matmul(W_neg.t())
        if bias is not None:
            lower = lower + bias
            upper = upper + bias
        return lower, upper

    def ibp_forward(self, bounds, **_):
        l, u = bounds.lower, bounds.upper
        B = l.shape[0]
        l = l.view(B, -1)
        u = u.view(B, -1)

        l, u = self._linear_ibp(l, u, self.fc.weight, self.fc.bias)

        l = torch.clamp(l, min=0.0)
        u = torch.clamp(u, min=0.0)

        l, u = self._linear_ibp(l, u, self.fc2.weight, self.fc2.bias)

        return IntervalBounds(bounds.region, l, u)

    def crown_backward(self):
        raise NotImplementedError("not impl")


In [20]:
factory = BoundModelFactory()
factory.register(Normalize, BoundNormalize)
factory.register(Net, BoundNet)
bmodel = factory.build(model).eval().to(device)


In [21]:
from bound_propagation import HyperRectangle
import pandas as pd

def make_region(images, eps):
    lo = (images - eps).clamp(0.0, 1.0)
    hi = (images + eps).clamp(0.0, 1.0)
    return HyperRectangle(lo, hi)

@torch.inference_mode()
def evaluate_verified_accuracy_ibp(eps_list):
    device = next(model.parameters()).device
    model.eval()
    bmodel.eval()

    total = correct = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        probs = model(images)
        correct += (probs.argmax(1) == labels).sum().item()
        total += labels.numel()
    baseline_acc = 100.0 * correct / total

    results = []
    for eps in eps_list:
        verified = total = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            region = make_region(images, float(eps))
            out = bmodel.ibp(region)
            lower, upper = out.lower, out.upper

            other_upper = upper.clone()
            other_upper.scatter_(1, labels.view(-1,1), float("-inf"))
            true_lower = lower.gather(1, labels.view(-1,1)).squeeze(1)

            verified += (true_lower > other_upper.max(dim=1).values).sum().item()
            total += labels.numel()

        results.append({"epsilon": float(eps), "verified_accuracy(%)": 100.0 * verified / total})

    return baseline_acc, pd.DataFrame(results)


In [22]:
epsilons = torch.linspace(0.01, 0.10, 10).tolist()
baseline_acc, df = evaluate_verified_accuracy_ibp(epsilons)
print(f"Clean (unperturbed) test accuracy: {baseline_acc:.2f}%")
print(df.to_string(index=False, formatters={'epsilon': '{:.2f}'.format,
                                            'verified_accuracy(%)': '{:.2f}'.format}))


Clean (unperturbed) test accuracy: 93.43%
epsilon verified_accuracy(%)
   0.01                49.59
   0.02                 5.30
   0.03                 0.20
   0.04                 0.00
   0.05                 0.00
   0.06                 0.00
   0.07                 0.00
   0.08                 0.00
   0.09                 0.00
   0.10                 0.00
